# SOCIAL NETWORK ANALYSIS

* **Proje  Amacı** :Bu proje, sosyal ağ analizi üzerine odaklanarak, bir Graph Neural Network (GNN) modeli kullanarak sosyal ağdaki düğümler üzerinden bir öneri sistemi geliştirmeyi amaçlamakta . Yani ağa katılan herhangi bir kişi için en öncelikli 10 kişiyi tahmin edip öneren bir sistem olması amaçlanmaktadır.

## VERİSETİ HAKKINDA

* Bu proje içerisinde kullandığım verisetini **Stanford**'a ait şu siteden aldım "https://snap.stanford.edu/data/"
* Veri setinde **Facebook**'a ait veriler bulunmaktadır
* Veriseti içerisinde 5 tip dosya bulunmaktadır.Bunla
    * **circles** DosyalarBu dosya, ego kullanıcısının çemberlerini içerir. Her bir çember, kullanıcıların belirli bir grup içinde nasıl gruplandırıldığını gösterir. Bu çemberler, sosyal ağdaki toplulukları veya grupları analiz etmek için kullanılabilir.r.
  * **edges** Dosyaları:Bu dosya, kullanıcılar arasındaki bağlantıları (kenarları) içerir. Her satır, iki kullanıcı arasındaki bir bağlantıyı temsil eder. Örneğin, "236 186" satırı, 236 ve 186 numaralı iki kullanıcı arasında bir bağlantı olduğunu belirtir.
  * **egofeat** DosyalBu dosya, ego kullanıcısının özelliklerini içerir. Bu özellikler, ego kullanıcısının profiline özgüdür ve diğer kullanıcılarla karşılaştırmak için kullanılabilir.lir.
  * **feat** DosyalBu dosya, ağdaki her bir düğümün özelliklerini içerir. Özellikler, düğümlerin profilleriyle ilişkilendirilebilir ve kullanıcıların belirli özelliklerini temsil edebilir.erir.
  * **featnames** DosyBu dosya, özellik isimlerini içerir. Özelliklerin neyi temsil ettiğini anlamanıza yardımcı olabilir.çnodeId.edges: Bu dosya, ego ağının kenarlarını içerir. Her bir satırda, bir kenarın iki ucu belirtilir. Kenarlar, genellikle kullanıcıların arkadaşlık ilişkilerini temsil eder.
lir.ıklar.

## KOD

* GEREKLİ KÜTÜPHANELERİN IMPORT EDİLMESİ

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from itertools import combinations
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


* VERİSETİNİN İÇE AKTARILMASI

In [2]:
df = pd.read_excel('veri.xlsx')

df.columns = ['source', 'target']

df.head(5)

,source,target
0,122,285
1,24,346
2,271,304
3,176,9
4,130,329


* VERİSETİ İLE GRAF YAPISININ OLUŞTURULMASI

In [3]:
G = nx.Graph()

edges = df.values.tolist()

G.add_edges_from(edges)

mapping = {node: idx for idx, node in enumerate(G.nodes())}

G = nx.relabel_nodes(G, mapping)

edge_index = torch.tensor(list(G.edges)).t().contiguous()


* NEO4J BAĞLANTI BİLGİLERİ 

In [4]:
from neo4j import GraphDatabase
import pandas as pd

# Neo4J DB bağlantı bilgileri - şifremi yazmak istemediğim için none yazdım 
uri = "bolt://localhost:7687"
username = "none"
password = "none"

driver = GraphDatabase.driver(uri, auth=(username, password))


* NEO4J'YE GRAFIN AKTARILMASI

In [5]:
def create_session():
    return driver.session()

def create_graph_from_dataframe(df):
    with create_session() as session:
        for index, row in df.iterrows():
            source = row['source']
            target = row['target']
            session.run("MERGE (a:Person {id: $source}) "
                        "MERGE (b:Person {id: $target}) "
                        "MERGE (a)-[:FRIEND]->(b)",
                        source=source, target=target)
            

* VERİSETİNİN GNN MODELİ İÇİN HAZIRLANMASI

In [6]:
num_nodes = G.number_of_nodes()
num_features = 3  # Özellik sayısı
num_classes = 2   # Sınıf sayısı

x = torch.rand((num_nodes, num_features))
y = torch.randint(0, num_classes, (num_nodes,))

data = Data(x=x, edge_index=edge_index, y=y)

* EĞİTİM VE TEST OLARAK VERİ SETİNİ AYIRMA 

In [7]:
train_mask, test_mask = train_test_split(range(num_nodes), test_size=0.2, random_state=42)

train_mask = torch.tensor(train_mask, dtype=torch.long)

test_mask = torch.tensor(test_mask, dtype=torch.long)


* GNN MODELİNİN OLUŞTURULMASI

In [8]:
class GCN(nn.Module):
    def __init__(self, num_features, hidden_dim1, hidden_dim2, num_classes, dropout=0.1):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

* MODELE AİT PARAMETRELERİN AYARLANMASI 

In [58]:
model = GCN(num_features=num_features, hidden_dim1=64, hidden_dim2=64, num_classes=num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.06)
criterion = nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = (pred[test_mask] == data.y[test_mask]).sum()
    acc = int(correct) / len(test_mask)
    return acc, pred[test_mask]

* MODELİN EĞİTİLMESİ VE TEST EDİLMESİ

In [60]:
for epoch in range(1000):
    loss = train()
    acc, _ = test()
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

_, pred = test()
true_labels = data.y[test_mask].cpu().numpy()
pred_labels = pred.cpu().numpy()

conf_matrix = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(true_labels, pred_labels)
print("Classification Report:\n", class_report)

    

Epoch 10, Loss: 0.6905, Test Accuracy: 0.5522
Epoch 20, Loss: 0.6916, Test Accuracy: 0.5821
Epoch 30, Loss: 0.6889, Test Accuracy: 0.5821
Epoch 40, Loss: 0.6912, Test Accuracy: 0.4925
Epoch 50, Loss: 0.6912, Test Accuracy: 0.4925
Epoch 60, Loss: 0.6920, Test Accuracy: 0.5075
Epoch 70, Loss: 0.6921, Test Accuracy: 0.4776
Epoch 80, Loss: 0.6913, Test Accuracy: 0.4925
Epoch 90, Loss: 0.6908, Test Accuracy: 0.5075
Epoch 100, Loss: 0.6922, Test Accuracy: 0.4925
Epoch 110, Loss: 0.6919, Test Accuracy: 0.5075
Epoch 120, Loss: 0.6920, Test Accuracy: 0.5075
Epoch 130, Loss: 0.6920, Test Accuracy: 0.5075
Epoch 140, Loss: 0.6918, Test Accuracy: 0.4925
Epoch 150, Loss: 0.6904, Test Accuracy: 0.5224
Epoch 160, Loss: 0.6895, Test Accuracy: 0.5522
Epoch 170, Loss: 0.6909, Test Accuracy: 0.5522
Epoch 180, Loss: 0.6928, Test Accuracy: 0.4925
Epoch 190, Loss: 0.6902, Test Accuracy: 0.5075
Epoch 200, Loss: 0.6897, Test Accuracy: 0.4925
Epoch 210, Loss: 0.6932, Test Accuracy: 0.4925
Epoch 220, Loss: 0.691

*  TAHMİN SÜRECİNDE KULLANMAK İÇİN MODELİN KAYDEDİLMESİ

In [61]:
torch.save(model.state_dict(), 'model.pth')


* KAYDEDİLEN MODELİN YÜKLENMESİ

In [62]:
model.load_state_dict(torch.load('model.pth'))
model.eval()


GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 2)
)

* YENİ KİŞİ EKLENDİĞİNDE GRAFI GÜNCELLEYEN FONKSİYON 

In [63]:
def add_new_person(G, new_person, friends):
    G.add_node(new_person)
    for friend in friends:
        G.add_edge(new_person, friend)
    return G

* YENİ NODE İLE İLİŞKİLİ EDGE EKLEME 

In [64]:
# İki kişi arasında arkadaşlık ekleme fonksiyonu
def add_friendship(source, target):
    with create_session() as session:
        session.run("""
            MATCH (a:Person {id: $source})
            MATCH (b:Person {id: $target})
            CREATE (a)-[:FRIEND]->(b)
            """, source=source, target=target)

* EKLENEN YENİ KİŞİNİN DİNAMİK OLARAK NEO4J'YE AKTARILMASI 

In [65]:
# Yeni bir kişi ekleme fonksiyonu
def add_person(person_id):
    with create_session() as session:
        session.run("CREATE (:Person {id: $id})", id=person_id)
        

* EKLENEN YENİ KİŞİ İÇİN EN YÜKSEK TAHMİN DEĞERLERİNE SAHİP İLK 10 KİŞİNİN BULUNMASI

In [66]:
def find_similar_people(model, G, new_person, friends, top_k=10):
    G = add_new_person(G, new_person, friends)
    
    mapping = {node: idx for idx, node in enumerate(G.nodes())}
    G = nx.relabel_nodes(G, mapping)
    
    edge_index = torch.tensor(list(G.edges())).t().contiguous()
    
    x = torch.ones((len(G.nodes()), num_features))  # Özellik boyutunu doğru ayarlayın
    
    data = Data(x=x, edge_index=edge_index)
    
    with torch.no_grad():
        out = model(data.x, data.edge_index)
    
    new_person_idx = mapping[new_person]
    
    cos_sim = torch.nn.functional.cosine_similarity(out, out[new_person_idx].unsqueeze(0), dim=1)
    
    top_k_indices = cos_sim.argsort(descending=True)[1:top_k+1]
    top_k_names = [list(G.nodes())[idx] for idx in top_k_indices]
    
    return top_k_names

* TAHMİN MODELİNİN DENENMESİ

In [67]:
person = 286
friends = [81]

similar_people = find_similar_people(model, G, person, friends)

print(similar_people)


[195, 1, 3, 33, 64, 141, 75, 199, 288, 84]


* DİNAMİK OLARAK NEO4J'DEN NODE SİLME

In [68]:
# Bir kişiyi silme fonksiyonu
def remove_person(person_id):
    with create_session() as session:
        session.run("MATCH (p:Person {id: $id}) DETACH DELETE p", id=person_id)
        

* MEVCUT NODE İLE İLİŞKİLİ EDGE'LERİ SİLME 

In [69]:
# İki kişi arasındaki arkadaşlığı silme fonksiyonu
def remove_friendship(source, target):
    with create_session() as session:
        session.run("""
            MATCH (a:Person {id: $source})-[r:FRIEND]->(b:Person {id: $target})
            DELETE r
            """, source=source, target=target)
        

* NEO4 YAVAŞ ÇALIŞTIĞINDAN YENİ NODE'UN EKLENİP / EKLENMEDİĞİNİN KONTROLÜ - RETURN TRUE YA DA FALSE 

In [70]:
def check_person_exists(person_id):
    with create_session() as session:
        result = session.run("MATCH (p:Person {id: $id}) RETURN p", id=person_id)
        return result.single() is not None
    

* GRAF'IN NEO4'YE AKTARIMI 

In [71]:
# Veriyi yükleyin ve grafı Neo4j'ye aktarın
df = pd.read_excel('veri.xlsx')
df.columns = ['source', 'target']
create_graph_from_dataframe(df)


* DİNAMİK OLARAK YENİ NODE VE EDGE EKLEME 

In [72]:
add_person(1234)
add_friendship(1234, 5678)


* YUKARIDAKİ KİŞİNİN GRAF'A EKLENİP EKLENMEDİĞİNİN KONTROLÜ 

In [73]:
print(check_person_exists(12)) 

False


In [74]:
print(check_person_exists(1234)) 

True


C:\Users\Gamze\anaconda3\Lib\site-packages\neo4j\_sync\work\result.py:500: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "
